In [41]:
# XML exercise
## Using data in 'data/mondial_database.xml', 
## and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find
## 1. 10 countries with the lowest infant mortality rates
## 2. 10 cities with the largest population
## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
## 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [42]:
import pandas as pd
from xml.etree import ElementTree as ET
from lxml import objectify

In [43]:
#import the XML data with its path
path = './data/mondial_database.xml'
parsed = objectify.parse(open(path, encoding= 'utf8'))
root = parsed.getroot()

In [44]:
#Question 1           
#  10 countries with the lowest infant mortality rates
# Easiest way to do this is using a list of dictionary as shownn below. It's considered computationally faster because lookup is by the key. Another way is by using a list of tuples as shown in subsequent solutions.

In [45]:
data = []
for child in root.findall('country'):
    child_data = {}
    for child in child.getchildren():
        child_data[child.tag] = child.text
    data.append(child_data)
    
df = pd.DataFrame(data)
df.head()

,border,city,dependent,encompassed,ethnicgroup,gdp_agri,gdp_ind,gdp_serv,gdp_total,government,...,infant_mortality,inflation,language,localname,name,population,population_growth,province,religion,unemployment
0,nan,nan,nan,None,Greek,19.5,12,68.5,12800,parliamentary democracy,...,13.19,1.7,Greek,NaN,Albania,2800138,0.3,nan,Christian Orthodox,16.9
1,nan,nan,nan,None,Greek,3.5,16,80.5,243300,parliamentary republic,...,4.78,-0.8,Greek,Ελληνική Δημοκρατία,Greece,10816286,0.01,nan,Muslim,27.9
2,nan,nan,nan,None,Serb,10.2,27.5,62.3,10650,parliamentary democracy,...,7.9,2.8,Serbian,Република Македониjа,Macedonia,2059794,0.21,nan,Roman Catholic,28.6
3,nan,nan,nan,None,Croat,7.9,31.8,60.3,43680,republic,...,6.16,2.2,Roma,Република Србија,Serbia,7120666,-0.46,nan,Protestant,20.1
4,nan,nan,nan,None,Albanian,0.8,11.3,87.9,4518,republic,...,NaN,4,Albanian,Црна Гора,Montenegro,620029,-0.49,nan,Roman Catholic,19.1


In [46]:
infant= df.loc[: , ['name', 'infant_mortality']]
infant['infant_mortality'] = infant['infant_mortality'].astype('float64')
lowest= infant.sort_values(by= 'infant_mortality', ascending= True)
lowest.head(10)

,name,infant_mortality
38,Monaco,2
98,Japan,2
117,Bermuda,2
36,Norway,2
106,Singapore,3
37,Sweden,3
10,Czech Republic,3
78,Hong Kong,3
79,Macao,3
44,Iceland,3


In [47]:
#Question 2         
#  10 cities with the largest population

In [48]:
cities = []
for capital in root.iter('city'):
    y = capital.findall('name')[0]
    cities.append(str(y))
#print(cities)
    

In [49]:
ppn = []
for elem in root.iter('city'):        
    try:
        y = elem.population[-1]
    except AttributeError:
        y = '0' 
    ppn.append(int(y))
     

In [50]:
data = list(zip(cities, ppn))
df = pd.DataFrame(data, columns=['city', 'population'])
df.head()


,city,population
0,Tirana,418495
1,Shkodër,77075
2,Durrës,113249
3,Vlorë,79513
4,Elbasan,78703


In [11]:
sorted_df = df.sort_values(by= 'population', ascending= False)
sorted_df.head(10)

,city,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [12]:
#Question 3  
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [13]:
#This is one way to do this by calling the path to the sub-elements like in the case of ctry.population
#
#w = []
#for ctry in root.findall('country'):
    #latest = ctry.population[-1]
    #for eth in ctry.iter('ethnicgroup'):
        #s = eth.text, eth.attrib['percentage'], latest
        #w.append(s)


In [14]:
z = []
for x in root.findall('country'):
    y = x.findall('population')[-1]
    for s in x.iter('ethnicgroup'):
        yyy = s.text, s.attrib['percentage'], y.text
        z.append(yyy)
#print(z)

In [15]:
df0 = pd.DataFrame(z, columns=['Ethic Group', 'Raw Percentage', 'Population'])  #Sorry for typo in Ethnic Group. :(
df0.head()

,Ethic Group,Raw Percentage,Population
0,Albanian,95,2800138
1,Greek,3,2800138
2,Greek,93,10816286
3,Macedonian,64.2,2059794
4,Albanian,25.2,2059794


In [16]:
df0['Raw Percentage'] = df0['Raw Percentage'].astype('float64')
df0['Population'] = df0['Population'].astype('int64')


In [17]:
df0.head()


,Ethic Group,Raw Percentage,Population
0,Albanian,95.0,2800138
1,Greek,3.0,2800138
2,Greek,93.0,10816286
3,Macedonian,64.2,2059794
4,Albanian,25.2,2059794


In [18]:
df0['Actual Percentage'] = df0['Raw Percentage']/100   #converted percentage into fraction to enable me find part of a whole of the Ethnic roup
df0.head()

,Ethic Group,Raw Percentage,Population,Actual Percentage
0,Albanian,95.0,2800138,0.950
1,Greek,3.0,2800138,0.030
2,Greek,93.0,10816286,0.930
3,Macedonian,64.2,2059794,0.642
4,Albanian,25.2,2059794,0.252


In [19]:
df0['Actual Population'] = df0['Population'] * df0['Actual Percentage'] #Actual population of each ethnic group is percentage of each ethnic group in each country multiplied by the latest population 
df0.head()

,Ethic Group,Raw Percentage,Population,Actual Percentage,Actual Population
0,Albanian,95.0,2800138,0.950,2.660131e+06
1,Greek,3.0,2800138,0.030,8.400414e+04
2,Greek,93.0,10816286,0.930,1.005915e+07
3,Macedonian,64.2,2059794,0.642,1.322388e+06
4,Albanian,25.2,2059794,0.252,5.190681e+05


In [20]:
df01= df0.loc[:, ['Ethic Group', 'Actual Population']]
df01.head()

,Ethic Group,Actual Population
0,Albanian,2.660131e+06
1,Greek,8.400414e+04
2,Greek,1.005915e+07
3,Macedonian,1.322388e+06
4,Albanian,5.190681e+05


In [21]:
top_eth= df01.groupby(by= 'Ethic Group')['Actual Population'].sum().sort_values(ascending= False)

In [22]:
pd.set_option('display.float_format', lambda x: '%.0f' % x) #awesome :)

In [23]:
top_eth.head(10)

Ethic Group
Han Chinese   1245058800
Indo-Aryan     871815583
European       494872220
African        318325120
Dravidian      302713744
Mestizo        157734355
Bengali        146776917
Russian        131856996
Japanese       126534212
Malay          121993550
Name: Actual Population, dtype: float64

In [24]:
#Question 4 
# Name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [25]:
#4a
# I had to create a function here to save unnecessary typing, and call elements on it

In [26]:
def someFunction(x):
    a1 = []
    for elem in root.iter(x):
        b1 = str(elem.name), elem.attrib['country']
        a1.append(b1)
    return a1
   

In [27]:
df1 = pd.DataFrame(someFunction('river'), columns=['River', 'Country']) #function here was called passing in 'river'
df1.head()

,River,Country
0,Thjorsa,IS
1,Joekulsa a Fjoellum,IS
2,Glomma,N
3,Lagen,N
4,Goetaaelv,S


In [28]:
length = []
for l in root.iter('river'):        
    try:
        y = l.length
    except AttributeError:
        y = '0' 
    length.append(int(y))
#print(length)


In [29]:
df2 = pd.DataFrame(length, columns=['River Length'])
df2.head()

,River Length
0,230
1,206
2,604
3,322
4,93


In [30]:
longest_river = pd.concat([df1, df2], axis=1).sort_values(by='River Length', ascending = False)
longest_river.head(1)


,River,Country,River Length
174,Amazonas,CO BR PE,6448


In [31]:
#b

In [32]:
df3 = pd.DataFrame(someFunction('lake'), columns=['Lake', 'Country']) #function was also called here passing in 'lake'
df3.head()

,Lake,Country
0,Inari,SF
1,Oulujaervi,SF
2,Kallavesi,SF
3,Saimaa,SF
4,Paeijaenne,SF


In [33]:
largest = []
for size in root.iter('lake'):        
    try:
        y = size.area
    except AttributeError:
        y = '0' 
    largest.append(int(y))
#print(largest)


In [34]:
df4 = pd.DataFrame(largest, columns=['Area'])
df4.head()

,Area
0,1040
1,928
2,472
3,4370
4,1118


In [35]:
largest_lake = pd.concat([df3, df4], axis=1).sort_values(by='Area', ascending = False)
largest_lake.head(1)

,Lake,Country,Area
54,Caspian Sea,R AZ KAZ IR TM,386400


In [36]:
#c

In [37]:
heights= []
for height in root.iter('airport'):
    try:
        y = int(height.elevation)
    except:
        y = 0  
    heights.append(y)
#print(heights)

In [38]:
df5 = pd.DataFrame(heights, columns=['Height'])
df5.head()


,Height
0,977
1,1792
2,38
3,811
4,822


In [39]:
df6 = pd.DataFrame(someFunction('airport'), columns=['Airport', 'Country']) #function also called here passing in 'airport'
df6.head()

,Airport,Country
0,Herat,AFG
1,Kabul Intl,AFG
2,Tirana Rinas,AL
3,Cheikh Larbi Tebessi,DZ
4,Batna Airport,DZ


In [40]:
highest_airport = pd.concat([df6, df5], axis=1).sort_values(by='Height', ascending = False)
highest_airport.head(1)

,Airport,Country,Height
80,El Alto Intl,BOL,4063
